<a href="https://colab.research.google.com/github/Justin-Murphy-2003/Justin-Murphy-2003/blob/main/four-writing-files/WEEK_4_ADVANCED_SQL_AND_FILES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4 - Advanced SQL / Writing Files

In this week's lab, we are going to expand our Really Simple Social Network by adding another table that will record connections between people! Because after all, what good is a social network if you can't ... be social?

Afterwards, we will be in a good position to run a report on the data, and then we can sell it to marketers for a massive profit! Isn't it great how few people read the terms of service? `/s`

## Importing our database
Since we are using a new colab notebook, we will need to reimport our database. Check your D2L Submission if you didn't keep a copy, you should be able to redownload it there. Run the below code block and the file upload picker will appear, after uploading your Database file, check that it appears in the `Files` tab of colab.

Remember that you may need to update the listings before it will appear.

In [1]:
from google.colab import files
uploaded = files.upload()


Saving social_network.db to social_network.db


## Creating our relationship table

In the code block below, we will be creating a new table in our database for `relationships` with the following parameters:


1.   A primary key column called `id` with the `INTEGER` datatype that is `NOT NULL`
2.   A foreign key column called `user_id` that links to the `id` column of the `people` table with the `INTEGER` datatype that is `NOT NULL`
3.   Another foreign key column called `friend_id` that also links to the `id` column of the `people` table with the `INTEGER` datatype that is `NOT NULL`
4.   A `date_created` table with the `DATE` datatype that is `NOT NULL`

The Below codeblock will get you started:



In [2]:
import sqlite3

#Retreive the Connection object, rename the file to match your uploaded DB if necessary.
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#The Primary Key column has been provided for you,
#Replace '#!...' with the remaining column names, datatypes, and foreign key declarations.
#Use the Lecture notes and the previous lab if you are unsure.
createRelationshipsTable = """CREATE TABLE IF NOT EXISTS relationships (
                          id integer PRIMARY KEY NOT NULL,
                          user_id integer NOT NULL,
                          friend_id integer NOT NULL,
                          FOREIGN KEY (user_id) REFERENCES people (id),
                          FOREIGN KEY (friend_id) REFERENCES people (id)
                        );"""

#Once again, we will execute the query, persist the changes, and close our connection.
myCursor.execute(createRelationshipsTable)
myConnection.commit()
myConnection.close()

## Populating Relationship Data

We're not going to be picky about who knows who - so for the purposes of this lab, we're going to use our own version of an AI matchmaker, the `randrange()` method.

Complete the below codeblock, inserting 1000 new rows into the relationships table.
Use a random number between `1 - 1000` for the `user_id` and `friend_id` columns. This will link up to the `id` column of the `people` table and effectivly simulate a random network of relationships.

The below code block is purposely sparse, as you should be able to use the patterns you've identified in the previous lab to complete this task.

In [3]:
import sqlite3
from datetime import datetime #For generating dates and times
from random import randrange #For generating random numbers within a range

#Retreive the Connection object, rename the file to match your uploaded DB if necessary.
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#Complete the below query
addRelationshipQuery =  """INSERT INTO relationships (user_id, friend_id)
                            VALUES (?, ?);"""

for _ in range(1000):
  #Populate the arguments that will be used in the query
  user_id = randrange(1, 1001)

  friend_id = user_id

  while friend_id == user_id:
    friend_id = randrange(1, 1001)

  args = (user_id, friend_id)
  #Execute the query
  myCursor.execute(addRelationshipQuery, args)

#Commit your changes and close the connection.
myConnection.commit()
myConnection.close()


In [4]:
import sqlite3
from pprint import pprint

myConnection = sqlite3.connect('social_network.db')
selectStatement = """SELECT * FROM relationships
                      LIMIT 20;"""
myCursor = myConnection.cursor()

myCursor.execute(selectStatement)
results = myCursor.fetchall()
pprint(results)

myCursor.close()

[(1, 84, 528),
 (2, 858, 550),
 (3, 130, 228),
 (4, 827, 300),
 (5, 80, 217),
 (6, 498, 696),
 (7, 694, 419),
 (8, 900, 398),
 (9, 982, 241),
 (10, 441, 454),
 (11, 669, 228),
 (12, 943, 884),
 (13, 806, 188),
 (14, 487, 505),
 (15, 746, 907),
 (16, 694, 398),
 (17, 17, 200),
 (18, 582, 715),
 (19, 944, 157),
 (20, 315, 703)]


## Building our Report

Now that our AI Matchmaker has generated our network of relationships, a marketing firm is offering us boatloads of cash for information about the amount of friends the users of our social network have.

The request is to return the name of each user, and the total number of relationships they have made.

If this was explained in awkward pseudo code, it might go something like this:
```
SELECT all relationships
JOIN them to the people they belong to
GROUP the results by the person's ID
return the person's name and COUNT of their relationships
```

After you have collected the data, you will need to write it to a `.txt` file. 

**Your submission will include the report `.txt` file, the copy of your updated `social_network.db` file, and a link to the colab notebook.**

Look at the aggregate functions from Week 3 and the Join examples from Week 4. There are a few different ways that you can structure a query to reach the correct answer.

**At a minimum, you will need to use a `LEFT JOIN` statement, a `GROUP BY` statement, and a `COUNT()` aggregate function.**

The below code block will get you started:

In [5]:
import sqlite3
from pprint import pprint

#Retreive the Connection object, rename the file to match your uploaded DB if necessary.
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#Complete the below query
query =  """ SELECT people.name, COUNT(relationships.friend_id) FROM relationships
            LEFT JOIN people ON relationships.user_id = people.id
            GROUP BY relationships.user_id;
         """

#Uncomment the below line after you have determined what the correct mode will be for your file pointer.
fp = open('/content/report.txt', mode='w+')

myCursor.execute(query)
results = myCursor.fetchall()
#pprint(results)

for result in results:
  #You will want to write the results to the file pointer here.
  fp.write(str(result[0])+" "+str(result[1]))
  #Use the newline character \n to create a line break in each write() statement.
  fp.write('\n')
  #Use print() statements as sanity checks if you are unsure of the integrity of your data.
  print(str(result[0])+" "+str(result[1]))
#Don't forget to close the file pointer after you have finished!
fp.close()

Mario Russell 1
Michael Stone 1
Tabitha Green 1
Stephen Cobb 1
Paige Oneal 1
Steven Moore 3
Sheila Humphrey 4
Jennifer Schmidt 2
Leonard Cook 1
Daniel Morales 5
Angela Thomas 2
Jeffery Buckley 1
Kim Miller 3
Mrs. Cynthia Kelly 1
John Bradley 2
Kurt Ryan 1
Kathryn Mclaughlin 1
Daniel Fox 1
Reginald Evans 3
Emily Harrell 1
Krystal May 1
Michelle Pitts 1
Adrian Ortiz 2
James Johnson 1
Carrie Nelson 2
Victoria Woods 3
Kimberly Ramirez 1
Kayla Reyes 2
Mr. Brandon Henry 2
Tyler Aguilar 2
Jason Dennis 1
Samantha Morales 1
Cassandra Walters 1
Annette Rivera 3
Anne Baker 1
Luis Hunter 1
Brandon Sanchez 1
Vanessa Maldonado 2
Tracy Freeman 1
Lisa Powell 2
Jennifer Guzman 2
Jamie Ramsey 1
Makayla Rodriguez 1
Carlos Carter 1
Ronald Smith MD 3
Ms. Tamara Walker 2
Timothy Herrera 2
Dana Cortez 2
Andrea Miller 2
Alyssa Simmons 2
Michelle Nolan 3
Andrew Hurst 2
Preston Cox 1
Anthony Johnson 1
Jessica Collins 1
Rebecca Walsh 2
Scott Martin 1
John Roberts 1
Jeffrey James 1
Stephen Miller 2
Mr. Thomas Dav

### Challenge: Write a CSV file

**This challenge isn't for extra marks, it's an opportunity for you to engage in some self guided discovery, only if you feel like it, and only if you have time.**

If you want to challenge yourself, alter the report so that it conforms with the `.csv` file type. There are a number of ways to do this, you can use the `Pandas` module, the `csv` module, or manually.